In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.prepare_data as pd
import utils.xyzuvd as xyzuvd
import utils.visual as visual
import utils.error as error
from utils.directory import DATA_DIR, DATASET_DIR

In [ ]:
epoch = 14
exp = 'exp4'

# Train

In [ ]:
data_split = 'train'
save_prefix = '%s_fpha' %data_split
keys = pd.get_keys(save_prefix)
xyz_gt = np.array(pd.read_all_lmdb_from_name(keys, save_prefix, 'xyz_gt', 'float32', (21, 3)))
uvd_gt_resize = np.array(pd.read_all_lmdb_from_name(keys, save_prefix, 'uvd_gt_resize', 'float32', (21, 3)))

uvd_gt = xyzuvd.xyz2uvd_color(xyz_gt)
REORDER = visual.REORDER

pred_file = os.path.join(DATA_DIR, exp, 'predict_%s_%s_uvd.txt' %(epoch, data_split))
pred_uvd_resize = pd.read_predict(pred_file)
pred_uvd_resize = np.reshape(pred_uvd_resize, (-1, 21, 3))
pred_uvd = pd.scale_annot_wh(pred_uvd_resize, (416, 416), (1920, 1080))
pred_xyz = xyzuvd.uvd2xyz_color(pred_uvd)
pred_file_845 = os.path.join(DATA_DIR, exp, 'predict_%s_%s_uvd_845.txt' %(epoch, data_split))
pred_uvd_resize_845 = pd.read_predict(pred_file_845)
pred_uvd_resize_845 = np.reshape(pred_uvd_resize_845, (-1, 845, 21, 3))

In [ ]:
print('%s UVD RESIZED mean_l2_error: ' %data_split, error.mean_pose_error(uvd_gt_resize, pred_uvd_resize))
print('%s UVD mean_l2_error: ' %data_split, error.mean_pose_error(uvd_gt, pred_uvd))
print('%s XYZ mean_l2_error: ' %data_split, error.mean_pose_error(xyz_gt, pred_xyz))

In [ ]:
idx = 0
file_name_i = keys[idx]
img = pd.get_img(file_name_i)
pred_uvd_i = pred_uvd[idx]
uvd_gt_i_resize = uvd_gt[idx]

fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(img)
ax[0].set_title('pred')
visual.visualize_joints_2d(ax[0], pred_uvd_i[REORDER], joint_idxs=False)
ax[1].imshow(img)
ax[1].set_title('true')
visual.visualize_joints_2d(ax[1], uvd_gt_i_resize[REORDER], joint_idxs=False)

In [ ]:
pred_conf_file = os.path.join(DATA_DIR, exp, 'predict_%s_%s_conf.txt' %(epoch, data_split))
pred_conf = pd.read_predict(pred_conf_file)
pred_conf = np.array(pred_conf)

pred_conf_i = pred_conf[idx]
norm_pred_conf_i = (pred_conf_i - np.amin(pred_conf_i))/(np.amax(pred_conf_i) - np.amin(pred_conf_i))
print('pred conf: ', np.argmax(pred_conf_i), pred_conf[idx, np.argmax(pred_conf_i)])

In [ ]:
max_idx = []
for i in range(len(norm_pred_conf_i)//5):
    max_idx.append(i*5 + np.argmax(norm_pred_conf_i[i*5:i*5+5]))
show_uvd = pred_uvd_resize_845[idx, np.argmax(pred_conf_i)]
img_rsz = pd.sk_resize(img, (416, 416))
import matplotlib.patches as patches
fig, ax = plt.subplots(figsize=(5,5))
visual.visualize_joints_2d(ax, show_uvd[REORDER], joint_idxs=False)
ax.imshow(img_rsz.astype('uint32'))
for i in range(len(max_idx)):
    index = np.unravel_index(i, (13, 13))
    x = index[0]
    y = index[1]
    al = norm_pred_conf_i[max_idx[i]]
    if al == np.amax(norm_pred_conf_i):
        c = 'r'
    elif al <= 0.9:
        c = 'b'
    else:
        c = 'y'
    rect = patches.Rectangle((x*32,y*32),32,32,linewidth=1, edgecolor=c, facecolor=c, fill=True, alpha=al)
    ax.add_patch(rect)

# Test

In [ ]:
data_split = 'test'
save_prefix = '%s_fpha' %data_split
keys = pd.get_keys(save_prefix)
xyz_gt = np.array(pd.read_all_lmdb_from_name(keys, save_prefix, 'xyz_gt', 'float32', (21, 3)))
uvd_gt_resize = np.array(pd.read_all_lmdb_from_name(keys, save_prefix, 'uvd_gt_resize', 'float32', (21, 3)))


uvd_gt = xyzuvd.xyz2uvd_color(xyz_gt)
REORDER = visual.REORDER

pred_file = os.path.join(DATA_DIR, exp, 'predict_%s_%s_uvd.txt' %(epoch, data_split))
pred_uvd_resize = pd.read_predict(pred_file)
pred_uvd_resize = np.reshape(pred_uvd_resize, (-1, 21, 3))
pred_uvd = pd.scale_annot_wh(pred_uvd_resize, (416, 416), (1920, 1080))
pred_xyz = xyzuvd.uvd2xyz_color(pred_uvd)
pred_file_845 = os.path.join(DATA_DIR, exp, 'predict_%s_%s_uvd_845.txt' %(epoch, data_split))
pred_uvd_resize_845 = pd.read_predict(pred_file_845)
pred_uvd_resize_845 = np.reshape(pred_uvd_resize_845, (-1, 845, 21, 3))


In [ ]:
print('%s UVD RESIZED mean_l2_error: ' %data_split, error.mean_pose_error(uvd_gt_resize, pred_uvd_resize))
print('%s UVD mean_l2_error: ' %data_split, error.mean_pose_error(uvd_gt, pred_uvd))
print('%s XYZ mean_l2_error: ' %data_split, error.mean_pose_error(xyz_gt, pred_xyz))

In [ ]:
idx = 0
file_name_i = keys[idx]
img = pd.get_img(file_name_i)
pred_uvd_i = pred_uvd[idx]
uvd_gt_i_resize = uvd_gt[idx]

fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(img)
ax[0].set_title('pred')
visual.visualize_joints_2d(ax[0], pred_uvd_i[REORDER], joint_idxs=False)
ax[1].imshow(img)
ax[1].set_title('true')
visual.visualize_joints_2d(ax[1], uvd_gt_i_resize[REORDER], joint_idxs=False)

In [ ]:
pred_conf_file = os.path.join(DATA_DIR, exp, 'predict_%s_%s_conf.txt' %(epoch, data_split))
pred_conf = pd.read_predict(pred_conf_file)
pred_conf = np.array(pred_conf)

pred_conf_i = pred_conf[idx]
norm_pred_conf_i = (pred_conf_i - np.amin(pred_conf_i))/(np.amax(pred_conf_i) - np.amin(pred_conf_i))
print('pred conf: ', np.argmax(pred_conf_i), pred_conf[idx, np.argmax(pred_conf_i)])

In [ ]:
max_idx = []
for i in range(len(norm_pred_conf_i)//5):
    max_idx.append(i*5 + np.argmax(norm_pred_conf_i[i*5:i*5+5]))
show_uvd = pred_uvd_resize_845[idx, np.argmax(pred_conf_i)]
img_rsz = pd.sk_resize(img, (416, 416))
import matplotlib.patches as patches
fig, ax = plt.subplots(figsize=(5,5))
visual.visualize_joints_2d(ax, show_uvd[REORDER], joint_idxs=False)
ax.imshow(img_rsz.astype('uint32'))
for i in range(len(max_idx)):
    index = np.unravel_index(i, (13, 13))
    x = index[0]
    y = index[1]
    al = norm_pred_conf_i[max_idx[i]]
    if al == np.amax(norm_pred_conf_i):
        c = 'r'
    elif al <= 0.9:
        c = 'b'
    else:
        c = 'y'
    rect = patches.Rectangle((x*32,y*32),32,32,linewidth=1, edgecolor=c, facecolor=c, fill=True, alpha=al)
    ax.add_patch(rect)